In [64]:
import os
os.chdir("/Users/finlayduff/Documents/BATH MSc/Dissertation")
from utils.data.results import load_combined_results
import pandas as pd

In [83]:
dataset_name = "recovery-news-data_None"
experiment_id = "eca18c07-5a96-44a7-94d9-e8ef59644c33"
df = load_combined_results(dataset_name=dataset_name, experiment_id=experiment_id)
df = df.join(pd.json_normalize(df['captured_credibility_signals']))

In [84]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

###############################################################################
# 1.  Condensed credibility‑signal definitions
###############################################################################

@dataclass(frozen=True)
class Signal:
    """Metadata for a signal extracted by the LLM."""
    name: str
    risk: bool  # True if presence indicates *lower* credibility


SIGNALS: List[Signal] = [
    # factual‑evidence signals (presence = good)
    Signal("evidence_present", False),
    Signal("explicit_unverified_claim", True),
    Signal("inference_error", True),

    # source‑quality signals
    Signal("credible_sourcing", False),
    Signal("external_corroboration", False),

    # style / tone risk signals
    Signal("strong_framing_tone", True),
    Signal("clickbait_headline", True),
    Signal("writing_quality_alert", True),
]

SIGNAL_INDEX: Dict[str, int] = {s.name: i for i, s in enumerate(SIGNALS)}
FEATURE_DIM: int = len(SIGNALS)

###############################################################################
# 2.  Vectorisation helpers
###############################################################################

SignalResult = Tuple[bool, float]  # (label_is_true, confidence)
SignalDict = Dict[str, SignalResult]


def signals_to_features(sig_dict: SignalDict) -> np.ndarray:
    """Convert a *single* article's signals into a feature vector (1 x d)."""
    vec = np.zeros(FEATURE_DIM, dtype=np.float32)
    for name, (is_true, conf) in sig_dict.items():
        idx = SIGNAL_INDEX.get(name)
        if idx is None:
            continue  # ignore unknown signals
        polarity = -1 if SIGNALS[idx].risk else 1  # invert risk signals
        vec[idx] = polarity * conf if is_true else 0.0
    return vec


def batch_to_matrix(batch: List[SignalDict]) -> np.ndarray:
    """Stack many article‑level dicts into an (n x d) matrix."""
    return np.vstack([signals_to_features(b) for b in batch])

def dataframe_to_matrix(df: pd.DataFrame) -> np.ndarray:
    """Convert a pandas DataFrame with Boolean columns into (n x d) matrix."""
    arr = np.zeros((len(df), FEATURE_DIM), dtype=np.float32)
    for sig in SIGNALS:
        if sig.name in df.columns:
            values = df[sig.name].astype(float).to_numpy()
            polarity = -1 if sig.risk else 1
            arr[:, SIGNAL_INDEX[sig.name]] = polarity * values
    return arr

###############################################################################
# 3.  Model helpers
###############################################################################

def _make_calibrator(base: LogisticRegression) -> CalibratedClassifierCV:
    """Return a calibrated classifier compatible with any sklearn version."""
    try:
        # sklearn ≥ 1.1 (base_estimator removed in 1.3)
        return CalibratedClassifierCV(estimator=base, method="sigmoid", cv=5)
    except TypeError:
        # fallback for older versions
        return CalibratedClassifierCV(base_estimator=base, method="sigmoid", cv=5)


def build_classifier() -> CalibratedClassifierCV:
    base = LogisticRegression(max_iter=1000, class_weight="balanced")
    return _make_calibrator(base)


def fit_classifier(X: np.ndarray, y: np.ndarray) -> CalibratedClassifierCV:
    clf = build_classifier()
    clf.fit(X, y)
    return clf


def fit_classifier_from_dataframe(X_df: pd.DataFrame, y_series: pd.Series) -> CalibratedClassifierCV:
    X_mat = dataframe_to_matrix(X_df)
    return fit_classifier(X_mat, y_series.to_numpy())


def predict_proba(clf: CalibratedClassifierCV, sig_dict: SignalDict) -> float:
    X = signals_to_features(sig_dict).reshape(1, -1)
    return float(clf.predict_proba(X)[0, 1])


def predict_proba_df(clf: CalibratedClassifierCV, row: pd.Series) -> float:
    X = dataframe_to_matrix(row.to_frame().T)
    return float(clf.predict_proba(X)[0, 1])


def save_model(clf: CalibratedClassifierCV, path: str | Path) -> None:
    joblib.dump(clf, Path(path))


def load_model(path: str | Path) -> CalibratedClassifierCV:
    return joblib.load(Path(path))

In [85]:
from config.signals import CREDIBILITY_SIGNALS_CONDENSED
training_columns = CREDIBILITY_SIGNALS_CONDENSED.keys()
feature_columns = [f"{col}.label" for col in training_columns]
from sklearn.model_selection import train_test_split


X = df[feature_columns]
y = df["actual"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [86]:
clf = fit_classifier_from_dataframe(X_train, y_train)

In [87]:
clf.predict(dataframe_to_matrix(X_test))

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    brier_score_loss, precision_recall_curve, auc
)

# if you did a train/test split
y_score = clf.predict_proba(dataframe_to_matrix(X_test))[:, 1]
y_pred  = (y_score >= 0.5).astype(int)          # default 0.5 threshold

print(classification_report(y_test, y_pred, digits=3))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

print("ROC AUC :", roc_auc_score(y_test, y_score))
print("Brier   :", brier_score_loss(y_test, y_score))

# Precision–recall AUC (often more informative if the classes are imbalanced)
prec, rec, _ = precision_recall_curve(y_test, y_score)
print("PR  AUC :", auc(rec, prec))


              precision    recall  f1-score   support

           0      0.000     0.000     0.000       132
           1      0.672     1.000     0.804       271

    accuracy                          0.672       403
   macro avg      0.336     0.500     0.402       403
weighted avg      0.452     0.672     0.541       403

Confusion matrix:
 [[  0 132]
 [  0 271]]
ROC AUC : 0.5
Brier   : 0.22025984708492638
PR  AUC : 0.8362282878411911


/Users/finlayduff/.pyenv/versions/3.10.4/envs/fakenews_venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/finlayduff/.pyenv/versions/3.10.4/envs/fakenews_venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/finlayduff/.pyenv/versions/3.10.4/envs/fakenews_venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

In [ ]:
classification_report()